In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import DataFrame,Series
import sys
import os

import matplotlib.pyplot as plt
%matplotlib inline       
import matplotlib


In [ ]:
def VideoAnalysisTimeRangeMonitor (subject, current_advanced, pre_post, session):
    file_path="C:/data/R03/"+"Video_Analysis_R03_"+current_advanced+".xlsx" #read in the video analysis file
    sheetName= subject+"_"+pre_post+"_"+current_advanced

    VA=pd.DataFrame(pd.read_excel(file_path,sheet_name=sheetName))
    #automatically divide the subject data into different sessions
    divide=[0]

    for i in range(len(VA["Start Time"])-1):

        d=VA["Start Time"][i+1]-VA["Start Time"][i]
        if d<=0:
            divide.append(i+1)
        d=0
    divide.append(len(VA["Start Time"])+1)
    s=int(session[-1])
    a=divide[s-1]
    b=divide[s]
    
    VA_END=VA['END_SECONDS']*1000 #convert seconds to millisecond
    VA_START=VA['START_SECONDS']*1000
    VA_END=VA_END[a:b] 
    VA_START=VA_START[a:b]
    TASK_MONITOR=VA['TASK_MONITOR'][a:b] #session1
    return VA_START, VA_END, TASK_MONITOR


In [ ]:
def RawDataVideoTime(subject,pre_post,session, current_advanced):
    file_path="C:/data/R03/RAW_DATA/"+current_advanced+"/"+subject+"_"+pre_post+".xlsx"  #read in the raw data only including 'Video time'
    sheetName= subject+"_"+pre_post+"_"+session

    RD=pd.DataFrame(pd.read_excel(file_path, sheet_name=sheetName, usecols=['Video_Time', 'Monitors']) )
    VT=RD["Video_Time"]
    VT_MS=[]
    VT_TIME=[]


    for i in range(len(VT)):
        # convert h:m:s:ms format to the number of milliseconds to align with the VideoAnalysis time range
        VT_MS=VT[i].split(':')

        N=int(VT_MS[0])*3600000+int(VT_MS[1])*60000+int(VT_MS[2])*1000+int(VT_MS[3]) 
        VT_TIME.append(N)
        N=0
    RD["Video_time_ms"]=VT_TIME
    return RD

In [ ]:
def mapping (subject, current_advanced, pre_post, session):
    MM=RawDataVideoTime(subject, pre_post, session, current_advanced) #MM stands for MONITORMAPPING
    MM["VA_START"], MM["VA_END"], MM["TASK_MONITOR"]=VideoAnalysisTimeRangeMonitor (subject, current_advanced, pre_post,session)
   
    for i in range(len(MM["Video_time_ms"])):
        for j in range(len(MM["VA_START"])):
            if MM["Video_time_ms"][i]>=MM["VA_START"][j] and MM["Video_time_ms"][i]<=MM["VA_END"][j]:
                MM["Monitors"][i]=MM["TASK_MONITOR"][j]
                break
    MM['Monitors']=MM['Monitors'].fillna(0) # 0 indicats that the participants are not looking at any monitors
    file_path_done="C:/data/R03/MAPPING_RESULTS/"+subject+current_advanced+"_"+pre_post+"_"+session+".xlsx"
    MM.to_excel(file_path_done)

    

In [ ]:
CURRENT=["S#2", "S#7", "S#8", "S#11", "S#12", "S#14", "S#15"]
ENHANCED=["S#17", "S#18", "S#19", "S#20", "S#21", "S#22"]
SN1_PRE=["S5", "S6"]
SN2_PRE=["S1", "S2", "S3", "S4", "S5", "S6" ]
SN2_POST=["S1"]
SN7_PRE=["S1", "S2", "S3"]
SN7_POST=["S1"]
SN8_PRE=["S1", "S2", "S3", "S4" ]
SN8_POST=["S1"]
SN11_PRE=["S1", "S2", "S3", "S4" ]
SN11_POST=["S1"]
SN12_PRE=["S1", "S2", "S3", "S4", "S5", "S6", "S7" ]
SN12_POST=["S1", "S2"]
SN14_PRE=["S1", "S2", "S3", "S4", "S5", "S6" ]
SN14_POST=["S1"]
SN15_POST=["S1"]
PRE_POST=["PRE", "POST"]
SEN17_POST=["S1", "S2", "S3"]
SEN18_POST=["S1"]
SEN19_POST=["S1"]
SEN20_POST=["S1"]
SEN22_POST=["S1"]
SEN21_POST=["S1", "S2"]
# SEN23_POST=["S1", "S2", "S3"]


for i in CURRENT:
    n=i.split("#") 
    n=n[1]
    for j in PRE_POST:
        s="SN"+n+"_"+j
        s=vars()[s]
        for k in s:
            try:
                mapping(i, "CURRENT", j, k)
            except:
                print ('fail')

for e in ENHANCED:
    n=e.split("#") 
    n=n[1]
    
    s="SEN"+n+"_"+"POST"
    s=vars()[s]
    for k in s:
        try:
            mapping(e, "ENHANCED", "POST", k)
        except:
            print ('fail')



In [ ]:
CURRENT=["S#1","S#12", "S#15"]
ENHANCED=["S#17", "S#18", "S#19", "S#20", "S#21", "S#22"]
SN1_PRE=["S5", "S6"]
SN1_POST=[]
SN12_PRE=["S6", "S7" ]
SN12_POST=[]

SN15_POST=["S1"]
SN15_PRE=[]
PRE_POST=["PRE", "POST"]
SEN17_POST=["S1", "S2", "S3"]
SEN18_POST=["S1"]
SEN19_POST=["S1"]
SEN20_POST=["S1"]
SEN22_POST=["S1"]
SEN21_POST=["S1", "S2"]
# SEN23_POST=["S1", "S2", "S3"]


for i in CURRENT:
    n=i.split("#") 
    n=n[1]
    for j in PRE_POST:
        s="SN"+n+"_"+j
        s=vars()[s]
        if s:
            for k in s:
                try:
                    mapping(i, "CURRENT", j, k)
                except:
                    print (i, j, k)
                    print ('fail')

for e in ENHANCED:
    n=e.split("#") 
    n=n[1]
    
    s="SEN"+n+"_"+"POST"
    s=vars()[s]
    for k in s:
        try:
            mapping(e, "ENHANCED", "POST", k)
        except:
            print (e, k)
            print ('fail')

